In [1]:
import math
import json

In [2]:
qasm = '''

OPENQASM 2.0;
include "hqslib1.inc";
qreg q0[6];
qreg a0[2];


qreg q1[6];
qreg q2[6];


creg init0[2];
creg init1[2];

creg meas0[6];
creg meas1[6];

creg hcheck0[2];
creg hcheck1[2];
creg hcheck2[2];
creg hcheck3[2];

creg magic0[6];
creg magic1[6];
creg magic2[6];


creg ycheck0[1];
creg ycheck1[1];
creg ycheck2[1];
creg ycheck3[1];


creg xcheck0[2];
creg flags0[2];

creg scratch[1];
gate chh() a,b
{
   ry(-pi/4) b;
   cz a,b;
   ry(pi/4) b;
}




// Begin the first magic state prep + injection
ry(pi/4) q1[0];
y q1[0];


h q1[1];
sdg q1[1];

h q1[2];
h q1[4];
cx q1[2], q1[3];
cx q1[4], q1[5];
cx q1[2], q1[0];
cx q1[3], q1[1];
cx q1[0], q1[4];
cx q1[1], q1[5];
cx q1[4], q1[2];
cx q1[5], q1[3];




h q0[0];
cx q0[0], q0[1];


chh q0[0], q1[0];
cx q0[1], q1[1];
chh q0[1], q1[1];
chh q0[0], q1[2];
cx q0[1], q1[3];
chh q0[1], q1[3];
chh q0[0], q1[4];
cx q0[1], q1[5];
chh q0[1], q1[5];



cx q0[0], q0[1];
t q0[0];
h q0[0];



// the value of logical H is -1, but we will set all ancilla to the dark state before measurement
x q0[0];
measure q0[0] -> hcheck0[0];
reset q0[0];
measure q0[1] -> hcheck0[1];
reset q0[1];



h q0[0];
cy q0[0], q1[1];
cy q0[0], q1[3];
cy q0[0], q1[5];
h q0[0];
measure q0[0] -> ycheck0[0];
reset q0[0];


// Begin the first magic state prep + injection
ry(pi/4) q2[0];


h q2[1];
sdg q2[1];

h q2[2];
h q2[4];
cx q2[2], q2[3];
cx q2[4], q2[5];
cx q2[2], q2[0];
cx q2[3], q2[1];
cx q2[0], q2[4];
cx q2[1], q2[5];
cx q2[4], q2[2];
cx q2[5], q2[3];




h a0[0];
cx a0[0], a0[1];


chh a0[0], q2[0];
cx a0[1], q2[1];
chh a0[1], q2[1];
chh a0[0], q2[2];
cx a0[1], q2[3];
chh a0[1], q2[3];
chh a0[0], q2[4];
cx a0[1], q2[5];
chh a0[1], q2[5];



cx a0[0], a0[1];
t a0[0];
h a0[0];

measure a0 -> hcheck1;
reset a0;



h a0[0];
cy a0[0], q2[1];
cy a0[0], q2[3];
cy a0[0], q2[5];
h a0[0];
measure a0[0] -> ycheck1[0];
reset a0[0];



cy q2, q1;

sdg q2;
h q2;
measure q2 -> magic0;
reset q2;

scratch[0] = magic0[0]^magic0[2];
scratch[0] = scratch[0]^magic0[4];

if(scratch[0] == 0) ry(-pi/2) q1;

barrier q0, q1;





h q0[0];
cx q0[0], a0[0];
cx q0[0], q0[1];
cx q0[0], q0[4];
cx q0[0], q0[5];
cx q0[0], a0[0];
measure a0[0] -> init0[0];
reset a0[0];

h q0[2];
cx q0[2], a0[1];
cx q0[2], q0[3];
cx q0[2], q0[4];
cx q0[2], q0[5];
cx q0[2], a0[1];
measure a0[1] -> init0[1];
reset a0[1];



h q0;

// after doing the first -pi/4 rotation, we do the CZ gate
cz q0, q1;

// we measure q0 now
// the CH should send |+>|H-> to |->|H-> so we should measure 1 in the X basis
h q0;
measure q0 -> meas0;
reset q0;

barrier q0, q1, q2;



// Begin the second magic state prep + injection
ry(pi/4) q2[0];


h q2[1];
sdg q2[1];

h q2[2];
h q2[4];
cx q2[2], q2[3];
cx q2[4], q2[5];
cx q2[2], q2[0];
cx q2[3], q2[1];
cx q2[0], q2[4];
cx q2[1], q2[5];
cx q2[4], q2[2];
cx q2[5], q2[3];




h a0[0];
cx a0[0], a0[1];


chh a0[0], q2[0];
cx a0[1], q2[1];
chh a0[1], q2[1];
chh a0[0], q2[2];
cx a0[1], q2[3];
chh a0[1], q2[3];
chh a0[0], q2[4];
cx a0[1], q2[5];
chh a0[1], q2[5];



cx a0[0], a0[1];
t a0[0];
h a0[0];

measure a0 -> hcheck2;
reset a0;



h a0[0];
cy a0[0], q2[1];
cy a0[0], q2[3];
cy a0[0], q2[5];
h a0[0];
measure a0[0] -> ycheck2[0];
reset a0[0];



cy q2, q1;

sdg q2;
h q2;
measure q2 -> magic1;
reset q2;

scratch[0] = magic1[0]^magic1[2];
scratch[0] = scratch[0]^magic1[4];

if(scratch[0] == 1) ry(pi/2) q1;






//We extract X syndromes here to detect any Y errors
h q2[0];
cx q2[0], q1[0];
cx q2[0], q2[1];
cx q2[0], q1[1];
cx q2[0], q1[2];
cx q2[0], q2[1];
cx q2[0], q1[3];
h q2[0];

measure q2[0] -> xcheck0[0];
measure q2[1] -> flags0[0];



h q2[2];
cx q2[2], q1[2];
cx q2[2], q2[3];
cx q2[2], q1[3];
cx q2[2], q1[4];
cx q2[2], q2[3];
cx q2[2], q1[5];
h q2[2];
measure q2[2] -> xcheck0[1];
measure q2[3] -> flags0[1];




// Rotate back from |H-> to |1> so we can measure (prep the final magic state in q0 for this purpose)
ry(pi/4) q0[0];


h q0[1];
sdg q0[1];

h q0[2];
h q0[4];
cx q0[2], q0[3];
cx q0[4], q0[5];
cx q0[2], q0[0];
cx q0[3], q0[1];
cx q0[0], q0[4];
cx q0[1], q0[5];
cx q0[4], q0[2];
cx q0[5], q0[3];




h a0[0];
cx a0[0], a0[1];


chh a0[0], q0[0];
cx a0[1], q0[1];
chh a0[1], q0[1];
chh a0[0], q0[2];
cx a0[1], q0[3];
chh a0[1], q0[3];
chh a0[0], q0[4];
cx a0[1], q0[5];
chh a0[1], q0[5];



cx a0[0], a0[1];
t a0[0];
h a0[0];

measure a0 -> hcheck3;
reset a0;




h a0[0];
cy a0[0], q0[1];
cy a0[0], q0[3];
cy a0[0], q0[5];
h a0[0];
measure a0[0] -> ycheck3[0];
reset a0[0];


cy q0, q1;

sdg q0;
h q0;
measure q0 -> magic2;


scratch[0] = magic2[0]^magic2[2];
scratch[0] = scratch[0]^magic2[4];

if(scratch[0] == 0) ry(-pi/2) q1;

measure q1 -> meas1;
'''

In [3]:
with open("Logical+H-.json", "r") as file:
    jobs = json.load(file)

In [4]:
def checks(r,i):
    if (r['init0'][i] != '00' or r['init1'][i] != '00' or r['hcheck0'][i] != '00' or r['hcheck1'][i] != '00' or r['flags0'][i] != '00' or r['xcheck0'][i] != '00'):
        #print('fails here')
        #print(r['init0'][i], r['init1'][i], r['hcheck0'][i], r['hcheck1'][i], r['flags0'][i], r['xcheck0'][i])
        return False
    if (r['hcheck2'][i] != '00' or r['hcheck3'][i] != '00' or r['ycheck0'][i] != '0' or r['ycheck1'][i] != '0' or r['ycheck2'][i] != '0' or r['ycheck3'][i] != '0'):
        return False
    meas0 = r['meas0'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False
    meas0 = r['meas1'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False


    meas0 = r['magic0'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False

    meas0 = r['magic1'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False

    meas0 = r['magic2'][i]
    if (int(meas0[5]) + int(meas0[4]) + int(meas0[3]) + int(meas0[2]))%2 == 1:
        return False
    if (int(meas0[3]) + int(meas0[2]) + int(meas0[1]) + int(meas0[0]))%2 == 1:
        return False
    return True


#At the end, we got |->|H-> -> |->|1>
def success(r,i):
    meas0 = r['meas0'][i]
    meas1 = r['meas1'][i]
    return ((int(meas0[5]) + int(meas0[3]) + int(meas0[1]))%2 == 1) and ((int(meas1[5]) + int(meas1[3]) + int(meas1[1]))%2 == 1)

In [5]:
fails = 0 
totals = 0
shots = 0
for j in jobs:
    r = j['results']
    for i in range(0,len(r['hcheck0'])):
        shots += 1
        if(checks(r,i)):
            totals += 1
            if(not success(r,i)):
                fails += 1
print('failures:', fails, 'totals:', totals)
print('logical failure rate:', fails/totals)
print('total accept rate:', totals/shots, 'standard deviation:', math.sqrt((totals/shots)*(shots-totals))/shots)

failures: 3 totals: 6732
logical failure rate: 0.000445632798573975
total accept rate: 0.6732 standard deviation: 0.004690434521448946


In [6]:
p = fails/totals
n = totals
wilson = (1/(1 + 1/n))*(1/(2*n))*(math.sqrt(4*n*p*(1-p) + 1))
wilson_mean = 1/(1+ (1/n))*(p + (1/(2*n)))
print('logical failure rate confidence interval: (',  wilson_mean - wilson, ',',  wilson_mean + wilson, ')',)

logical failure rate confidence interval: ( 0.00025213057913387555 , 0.0007875248493526832 )
